# Introduction

<img src="img/tidbit2.png" width=300 style="float: right;"/>

This notebook demonstrates how BioThings Explorer can be used to answer the following query:

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;*"What drugs might be used to treat Parkinson's disease?"*

This query corresponds to [Tidbit 2](https://ncats.nih.gov/tidbit/tidbit_02.html) which was formulated as a demonstration of the NCATS Translator program.

**To experiment with an executable version of this notebook, [load it in Google Colaboratory](https://colab.research.google.com/github/biothings/biothings_explorer/blob/master/jupyter%20notebooks/TIDBIT%2002%20Finding%20New%20Uses%20for%20Existing%20Drugs%20to%20Treat%20Parkinson’s%20Disease.ipynb).**

**Background**: BioThings Explorer can answer two classes of queries -- "EXPLAIN" and "PREDICT".  EXPLAIN queries are described in [EXPLAIN_demo.ipynb](https://github.com/biothings/biothings_explorer/blob/master/jupyter%20notebooks/EXPLAIN_demo.ipynb), and PREDICT queries are described in [PREDICT_demo.ipynb](https://github.com/biothings/biothings_explorer/blob/master/jupyter%20notebooks/PREDICT_demo.ipynb). Here, we describe PREDICT queries and how to use BioThings Explorer to execute them.  A more detailed overview of the BioThings Explorer systems is provided in [these slides](https://docs.google.com/presentation/d/1QWQqqQhPD_pzKryh6Wijm4YQswv8pAjleVORCPyJyDE/edit?usp=sharing).

## Step 0: Load BioThings Explorer modules

Install the `biothings_explorer` and `biothings_schema` packages, as described in this [README](https://github.com/biothings/biothings_explorer/blob/master/jupyter%20notebooks/README.md#prerequisite).  This only needs to be done once (but including it here for compability with [colab](https://colab.research.google.com/)).

In [ ]:
!pip install git+https://github.com/biothings/biothings_explorer#egg=biothings_explorer
!pip install git+https://github.com/biothings/biothings_schema.py#egg=biothings_schema

Then import the relevant modules:

In [ ]:
# import modules from biothings_explorer
from biothings_explorer.hint import Hint
from biothings_explorer.user_query_dispatcher import FindConnection

## Step 1: Find representation of "Parkinson disease" in BTE

In this step, BioThings Explorer translates our query string "Parkinson disease"  into BioThings objects, which contain mappings to many common identifiers.  Generally, the top result returned by the `Hint` module will be the correct item, but you should confirm that using the identifiers shown.

Search terms can correspond to any child of [BiologicalEntity](https://biolink.github.io/biolink-model/docs/BiologicalEntity.html) from the [Biolink Model](https://biolink.github.io/biolink-model/docs/), including `DiseaseOrPhenotypicFeature` (e.g., "lupus"), `ChemicalSubstance` (e.g., "acetaminophen"), `Gene` (e.g., "CDK2"), `BiologicalProcess` (e.g., "T cell differentiation"), and `Pathway` (e.g., "Citric acid cycle").

In [1]:
from biothings_explorer.hint import Hint
ht = Hint()
parkDis = ht.query("Parkinson disease")['DiseaseOrPhenotypicFeature'][0]

parkDis

{'mondo': 'MONDO:0005180',
 'doid': 'DOID:14330',
 'umls': 'C0030567',
 'mesh': 'D010300',
 'name': 'Parkinson disease',
 'display': 'mondo(MONDO:0005180) doid(DOID:14330) umls(C0030567) mesh(D010300) name(Parkinson disease) ',
 'type': 'DiseaseOrPhenotypicFeature',
 'primary': {'identifier': 'mondo',
  'cls': 'DiseaseOrPhenotypicFeature',
  'value': 'MONDO:0005180'}}

## Step 2: Find drugs that are associated with genes which invovled in Parkinson disease

In this section, we find all paths in the knowledge graph that connect Parkinson disease to any entity that is a chemical compound.  To do that, we will use `FindConnection`.  This class is a convenient wrapper around two advanced functions for **query path planning** and **query path execution**. More advanced features for both query path planning and query path execution are in development and will be documented in the coming months. 

The parameters for `FindConnection` are described below:


In [2]:
from biothings_explorer.user_query_dispatcher import FindConnection

fc = FindConnection(input_obj=parkDis, output_obj='ChemicalSubstance', intermediate_nodes=['Gene'])
fc.connect(verbose=True)

========== QUERY PARAMETER SUMMARY ==========

BTE will find paths that join 'Parkinson disease' and 'ChemicalSubstance'. Paths will have 1 intermediate node.

Intermediate node #1 will have these type constraints: Gene




========== QUERY #1 -- fetch all Gene entities linked to Parkinson disease ==========

==== Step #1: Query path planning ====

Because Parkinson disease is of type 'DiseaseOrPhenotypicFeature', BTE will query our meta-KG for APIs that can take 'DiseaseOrPhenotypicFeature' as input

BTE found 5 apis:

API 1. mydisease.info(1 API call)
API 2. biolink_disease2gene(1 API call)
API 3. DISEASES(1 API call)
API 4. mgigene2phenotype(1 API call)
API 5. semmeddisease(1 API call)


==== Step #2: Query path execution ====
NOTE: API requests are dispatched in parallel, so the list of APIs below is ordered by query time.

API 1.1: http://mydisease.info/v1/query (POST "q=C0030567&scopes=mondo.xrefs.umls,disgenet.xrefs.umls&fields=disgenet.genes_related_to_disease&species=human&siz

API 1.15: https://platform-api.opentargets.io/v3/platform/public/evidence/filter?target=ENSG00000151006&datasource=chembl&size=15&fields=drug
API 1.22: https://platform-api.opentargets.io/v3/platform/public/evidence/filter?target=ENSG00000109743&datasource=chembl&size=15&fields=drug
robokop_gene2chemical failed
API 4.24: http://www.dgidb.org/api/v2/interactions.json?genes=PITX3
API 4.46: http://www.dgidb.org/api/v2/interactions.json?genes=UCN2
robokop_gene2chemical failed
robokop_gene2chemical failed
API 1.23: https://platform-api.opentargets.io/v3/platform/public/evidence/filter?target=ENSG00000185651&datasource=chembl&size=15&fields=drug
API 1.24: https://platform-api.opentargets.io/v3/platform/public/evidence/filter?target=ENSG00000131143&datasource=chembl&size=15&fields=drug
API 4.35: http://www.dgidb.org/api/v2/interactions.json?genes=TMEM175
robokop_gene2chemical failed
API 4.25: http://www.dgidb.org/api/v2/interactions.json?genes=CDNF
robokop_gene2chemical failed
API 4.30: http:

API 1.56: https://platform-api.opentargets.io/v3/platform/public/evidence/filter?target=ENSG00000125740&datasource=chembl&size=15&fields=drug
API 4.93: http://www.dgidb.org/api/v2/interactions.json?genes=CTC1
API 4.78: http://www.dgidb.org/api/v2/interactions.json?genes=FGB
API 1.57: https://platform-api.opentargets.io/v3/platform/public/evidence/filter?target=ENSG00000099365&datasource=chembl&size=15&fields=drug
robokop_gene2chemical failed
API 4.109: http://www.dgidb.org/api/v2/interactions.json?genes=SYT17
API 4.111: http://www.dgidb.org/api/v2/interactions.json?genes=HSPA9
API 1.54: https://platform-api.opentargets.io/v3/platform/public/evidence/filter?target=ENSG00000284320&datasource=chembl&size=15&fields=drug
API 1.58: https://platform-api.opentargets.io/v3/platform/public/evidence/filter?target=ENSG00000135018&datasource=chembl&size=15&fields=drug
API 1.59: https://platform-api.opentargets.io/v3/platform/public/evidence/filter?target=ENSG00000163069&datasource=chembl&size=15&fi

API 1.84: https://platform-api.opentargets.io/v3/platform/public/evidence/filter?target=ENSG00000107643&datasource=chembl&size=15&fields=drug
robokop_gene2chemical failed
API 4.88: http://www.dgidb.org/api/v2/interactions.json?genes=ALDH1A1
API 4.99: http://www.dgidb.org/api/v2/interactions.json?genes=TXNIP
robokop_gene2chemical failed
API 1.86: https://platform-api.opentargets.io/v3/platform/public/evidence/filter?target=ENSG00000124226&datasource=chembl&size=15&fields=drug
API 1.87: https://platform-api.opentargets.io/v3/platform/public/evidence/filter?target=ENSG00000284792&datasource=chembl&size=15&fields=drug
API 1.88: https://platform-api.opentargets.io/v3/platform/public/evidence/filter?target=ENSG00000145050&datasource=chembl&size=15&fields=drug
API 1.85: https://platform-api.opentargets.io/v3/platform/public/evidence/filter?target=ENSG00000167434&datasource=chembl&size=15&fields=drug
API 4.97: http://www.dgidb.org/api/v2/interactions.json?genes=MUL1
robokop_gene2chemical faile

API 4.186: http://www.dgidb.org/api/v2/interactions.json?genes=SH3BGRL
API 4.185: http://www.dgidb.org/api/v2/interactions.json?genes=SYNJ1
API 4.187: http://www.dgidb.org/api/v2/interactions.json?genes=DDIT4
robokop_gene2chemical failed
API 4.189: http://www.dgidb.org/api/v2/interactions.json?genes=MIR30E
API 1.124: https://platform-api.opentargets.io/v3/platform/public/evidence/filter?target=ENSG00000278174&datasource=chembl&size=15&fields=drug
API 1.123: https://platform-api.opentargets.io/v3/platform/public/evidence/filter?target=ENSG00000206287&datasource=chembl&size=15&fields=drug
robokop_gene2chemical failed
API 4.190: http://www.dgidb.org/api/v2/interactions.json?genes=CRMP1
API 1.125: https://platform-api.opentargets.io/v3/platform/public/evidence/filter?target=ENSG00000063176&datasource=chembl&size=15&fields=drug
robokop_gene2chemical failed
robokop_gene2chemical failed
API 4.178: http://www.dgidb.org/api/v2/interactions.json?genes=AKT1
robokop_gene2chemical failed
robokop_ge

...

API 4.455 dgidb_gene2chemical: No hits
API 4.456 dgidb_gene2chemical: No hits
API 4.457 dgidb_gene2chemical: No hits
API 4.458 dgidb_gene2chemical: 4 hits
API 4.459 dgidb_gene2chemical: No hits
API 4.460 dgidb_gene2chemical: No hits
API 4.461 dgidb_gene2chemical: No hits
API 4.462 dgidb_gene2chemical: 2 hits
API 4.463 dgidb_gene2chemical: No hits
API 4.464 dgidb_gene2chemical: No hits
API 4.465 dgidb_gene2chemical: 56 hits
API 4.466 dgidb_gene2chemical: No hits
API 4.467 dgidb_gene2chemical: No hits
API 4.468 dgidb_gene2chemical: 4 hits
API 4.469 dgidb_gene2chemical: No hits
API 4.470 dgidb_gene2chemical: 3 hits
API 4.471 dgidb_gene2chemical: No hits
API 4.472 dgidb_gene2chemical: No hits
API 4.473 dgidb_gene2chemical: No hits
API 4.474 dgidb_gene2chemical: No hits
API 4.475 dgidb_gene2chemical: No hits
API 4.476 dgidb_gene2chemical: 2 hits
API 4.477 dgidb_gene2chemical: 5 hits
API 4.478 dgidb_gene2chemical: 4 hits
API 4.479 dgidb_gene2chemical: No hits
API 4.480 dgidb_gene2chemical: N

API 1.90 opentarget: No hits
API 1.91 opentarget: No hits
API 1.92 opentarget: 2 hits
API 1.93 opentarget: No hits
API 1.94 opentarget: No hits
API 1.95 opentarget: No hits
API 1.96 opentarget: 15 hits
API 1.97 opentarget: No hits
API 1.98 opentarget: No hits
API 1.99 opentarget: No hits
API 1.100 opentarget: No hits
API 1.101 opentarget: No hits
API 1.102 opentarget: No hits
API 1.103 opentarget: No hits
API 1.104 opentarget: No hits
API 1.105 opentarget: No hits
API 1.106 opentarget: No hits
API 1.107 opentarget: No hits
API 1.108 opentarget: No hits
API 1.109 opentarget: 15 hits
API 1.110 opentarget: No hits
API 1.111 opentarget: No hits
API 1.112 opentarget: No hits
API 1.113 opentarget: No hits
API 1.114 opentarget: No hits
API 1.115 opentarget: No hits
API 1.116 opentarget: No hits
API 1.117 opentarget: No hits
API 1.118 opentarget: No hits
API 1.119 opentarget: No hits
API 1.120 opentarget: No hits
API 1.121 opentarget: No hits
API 1.122 opentarget: No hits
API 1.123 opentarget:

API 1.366 opentarget: No hits
API 1.367 opentarget: 15 hits
API 1.368 opentarget: No hits
API 1.369 opentarget: No hits
API 1.370 opentarget: No hits
API 1.371 opentarget: No hits
API 1.372 opentarget: No hits
API 1.373 opentarget: No hits
API 1.374 opentarget: No hits
API 1.375 opentarget: No hits
API 1.376 opentarget: No hits
API 1.377 opentarget: 9 hits
API 1.378 opentarget: 6 hits
API 1.379 opentarget: No hits
API 1.380 opentarget: 2 hits
API 1.381 opentarget: No hits
API 1.382 opentarget: No hits
API 1.383 opentarget: No hits
API 1.384 opentarget: 9 hits
API 1.385 opentarget: No hits
API 1.386 opentarget: 15 hits
API 1.387 opentarget: No hits
API 1.388 opentarget: 3 hits
API 1.389 opentarget: No hits
API 1.390 opentarget: No hits
API 1.391 opentarget: No hits
API 1.392 opentarget: 15 hits
API 1.393 opentarget: No hits
API 1.394 opentarget: No hits
API 1.395 opentarget: No hits
API 1.396 opentarget: No hits
API 1.397 opentarget: 15 hits
API 1.398 opentarget: No hits
API 1.399 opent

API 1.644 opentarget: No hits
API 1.645 opentarget: No hits
API 1.646 opentarget: No hits
API 1.647 opentarget: No hits
API 1.648 opentarget: No hits
API 1.649 opentarget: No hits
API 1.650 opentarget: No hits
API 1.651 opentarget: No hits
API 1.652 opentarget: No hits
API 1.653 opentarget: No hits
API 1.654 opentarget: No hits
API 1.655 opentarget: 15 hits
API 1.656 opentarget: No hits
API 1.657 opentarget: No hits
API 1.658 opentarget: No hits
API 1.659 opentarget: No hits
API 1.660 opentarget: No hits
API 1.661 opentarget: No hits
API 1.662 opentarget: No hits
API 1.663 opentarget: No hits
API 1.664 opentarget: 15 hits
API 1.665 opentarget: No hits
API 1.666 opentarget: No hits
API 1.667 opentarget: 15 hits
API 1.668 opentarget: No hits
API 1.669 opentarget: 3 hits
API 1.670 opentarget: No hits
API 1.671 opentarget: No hits
API 1.672 opentarget: No hits
API 1.673 opentarget: No hits
API 1.674 opentarget: 15 hits
API 1.675 opentarget: 15 hits
API 1.676 opentarget: No hits
API 1.677 o

API 2.1 semmedgene: 64913 hits

After id-to-object translation, BTE retrieved 12033 unique objects.

========== Final assembly of results ==========


In the #1 query, BTE found 757 unique Gene nodes
In the #2 query, BTE found 12033 unique ChemicalSubstance nodes


In [3]:
df = fc.display_table_view()

The df object contains the full output from BioThings Explorer. Each row shows one path that joins the input node (Parkinson's disease) to an intermediate node (a gene or protein) to an ending node (a chemical compound). The data frame includes a set of columns with additional details on each node and edge (including human-readable labels, identifiers, and sources). Let's remove all examples where the output_name (the compound label) is None, and specifically focus on paths with specific mechanistic predicates causedBy and targetedBy.

In [4]:
dfFilt = df.loc[df['output_name'].notnull()].query('pred1 == "causedBy" and pred2 == "targetedBy"')
dfFilt

,input,input_type,pred1,pred1_source,pred1_api,pred1_pubmed,node1_id,node1_name,node1_type,pred2,pred2_source,pred2_api,pred2_pubmed,output_id,output_name,output_type
117,Parkinson disease,DiseaseOrPhenotypicFeature,causedBy,semmed,semmeddisease,,entrez:1813,DRD2,Gene,targetedBy,mychem.info,mychem.info,None,chembl:CHEMBL231068,FLIBANSERIN,ChemicalSubstance
178,Parkinson disease,DiseaseOrPhenotypicFeature,causedBy,semmed,semmeddisease,,entrez:1565,CYP2D6,Gene,targetedBy,dgidb,dgidb_gene2chemical,None,chembl:CHEMBL654,MIRTAZAPINE,ChemicalSubstance
229,Parkinson disease,DiseaseOrPhenotypicFeature,causedBy,semmed,semmeddisease,,entrez:6571,SLC18A2,Gene,targetedBy,mychem.info,mychem.info,None,chembl:CHEMBL39,SEROTONIN,ChemicalSubstance
252,Parkinson disease,DiseaseOrPhenotypicFeature,causedBy,semmed,semmeddisease,,entrez:4544,MTNR1B,Gene,targetedBy,dgidb,dgidb_gene2chemical,None,chembl:CHEMBL33185,InChI=1S/C20H21ClN2O2/c1-3-20(24)22-12-16-11-1...,ChemicalSubstance
351,Parkinson disease,DiseaseOrPhenotypicFeature,causedBy,semmed,semmeddisease,,entrez:1813,DRD2,Gene,targetedBy,mychem.info,mychem.info,None,chembl:CHEMBL1541,CEFIXIME,ChemicalSubstance
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105410,Parkinson disease,DiseaseOrPhenotypicFeature,causedBy,semmed,semmeddisease,,entrez:1813,DRD2,Gene,targetedBy,mychem.info,mychem.info,None,chembl:CHEMBL2110775,CARPIPRAMINE,ChemicalSubstance
105435,Parkinson disease,DiseaseOrPhenotypicFeature,causedBy,semmed,semmeddisease,,entrez:1137,CHRNA4,Gene,targetedBy,dgidb,dgidb_gene2chemical,None,chembl:CHEMBL449,BUTABARBITAL,ChemicalSubstance
105436,Parkinson disease,DiseaseOrPhenotypicFeature,causedBy,semmed,semmeddisease,,entrez:1137,CHRNA4,Gene,targetedBy,mychem.info,mychem.info,None,chembl:CHEMBL449,BUTABARBITAL,ChemicalSubstance
105501,Parkinson disease,DiseaseOrPhenotypicFeature,causedBy,semmed,semmeddisease,,entrez:1813,DRD2,Gene,targetedBy,dgidb,dgidb_gene2chemical,None,chembl:CHEMBL2103832,PARDOPRUNOX,ChemicalSubstance


Let's examine how many unique Parkinsons - GENE - DRUG paths there are:

In [5]:
dfFiltUnique = dfFilt[["input","node1_name","output_name"]].drop_duplicates()
dfFiltUnique

,input,node1_name,output_name
117,Parkinson disease,DRD2,FLIBANSERIN
178,Parkinson disease,CYP2D6,MIRTAZAPINE
229,Parkinson disease,SLC18A2,SEROTONIN
252,Parkinson disease,MTNR1B,InChI=1S/C20H21ClN2O2/c1-3-20(24)22-12-16-11-1...
351,Parkinson disease,DRD2,CEFIXIME
...,...,...,...
105322,Parkinson disease,CHRNA4,ALCOHOL
105391,Parkinson disease,FYN,CHLORPROMAZINE
105410,Parkinson disease,DRD2,CARPIPRAMINE
105435,Parkinson disease,CHRNA4,BUTABARBITAL


## Results

Finally, let's examine the list of genes identified as intermediates between Parkinson's Disease and some chemical compound, sorted by the frequency of occurrence:

In [6]:
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_colwidth', 2000)

print("Num genes: "+str(len(dfFiltUnique["node1_name"].value_counts()))+"\n")
print(dfFiltUnique["node1_name"].value_counts())


Num genes: 75

DRD2       256
CA2        156
CYP2D6     151
SLC6A2     149
TP53       112
TNF        100
ADORA1      99
CHRNA4      82
BDNF        72
FYN         70
BAX         54
AKT1        49
APP         40
IL6         38
HSPA4       36
MAPK8       33
COMT        29
NFE2L2      29
SLC18A2     29
MTNR1B      28
TH          19
CXCR4       18
PDE10A      18
GSTM1       17
P2RX7       17
TAT         16
RHO         14
HSPA8       13
NAT2        13
EPHB1       13
MC1R        12
NR4A1       11
LRRK2       10
SIRT1        9
CAT          8
FXN          7
EGF          7
GBA          7
FTL          7
MAPT         6
LCN2         6
RPL3         6
GSN          5
VCP          5
TRH          5
PON1         5
MT2A         5
ATP7A        5
SOD2         5
DAPK2        5
CDK5R1       4
ADCYAP1      4
PSMC1        4
SNCA         3
CDC42        3
UTRN         3
PRDX2        3
NCAM1        2
GRK2         2
COX7B        2
GABPA        2
PAK1         2
COL11A2      2
COX4I1       2
CRMP1        1
PARK7     

Within this list, we found a number of genes associated with therapies for Parkinson's Disease that are either approved or in development, including **_DRD2_**, **_COMT_**, **_LRRK2_**, and **_SNCA_**.  We can also view the many chemical compounds that are known to modulate those genes using the query below:

In [7]:
knownPDgenes = dfFilt[dfFilt["node1_name"].isin(["LRRK2","COMT","DRD2","SNCA"])][["input","node1_name","output_name"]].drop_duplicates().sort_values("node1_name")
compounds = knownPDgenes.groupby(['node1_name'])['output_name'].apply(','.join)
count = knownPDgenes.groupby(['node1_name'])['output_name'].count()
result = pd.DataFrame({ 'count': count, 'compounds': compounds } )
result

,count,compounds
node1_name,,
COMT,29,"METHYLDOPA,CLIOQUINOL,ADEMETIONINE,NIALAMIDE,InChI=1S/C15H10O4/c16-12-7-6-10-11(9-4-2-1-3-5-9)8-13(17)19-15(10)14(12)18/h1-8,16,18H,ENTACAPONE,CARBOQUONE,TILIQUINOL,CHINIOFON,FLOPROPIONE,DOBUTAMINE,OXYQUINOLINE,InChI=1S/C20H20F3N3O5/c21-20(22,23)14-2-1-3-15(12-14)25-8-6-24(7-9-25)5-4-17(27)13-10-16(26(30)31)19(29)18(28)11-13/h1-3,10-12,28-29H,4-9H2,RISPERIDONE,BUPROPION,IODOQUINOL,LEVODOPA,NITROXOLINE,2-METHOXYESTRADIOL,D-PROLINE,TOLCAPONE,S-ADENOSYL-L-METHIONINE,OPICAPONE,InChI=1S/C19H19N7O8/c20-16-12-17(23-6-22-16)25(7-24-12)19-15(30)14(29)11(34-19)2-1-3-21-18(31)9-4-8(26(32)33)5-10(27)13(9)28/h1-2,4-7,11,14-15,19,27-30H,3H2,(H,21,31)(H2,20,22,23)/b2-1+/t11-,14-,15-,19-/m1/s1,CEFOTETAN,3,5-DINITROCATECHOL (OR-486),InChI=1S/C13H9NO5/c15-10-7-6-9(11(13(10)17)14(18)19)12(16)8-4-2-1-3-5-8/h1-7,15,17H,BROXYQUINOLINE,CLOXYQUIN"
DRD2,256,"PROCHLORPERAZINE EDISYLATE,ASENAPINE MALEATE,INDACATEROL,TRIMIPRAMINE MALEATE,FEMOXETINE,LONAPALENE,FLUPHENAZINE ENANTHATE,PRIDOPIDINE,GEPIRONE,LOXAPINE SUCCINATE,PRAMIPEXOLE,InChI=1S/C26H30F2N2O/c27-23-9-5-20(6-10-23)26(21-7-11-24(28)12-8-21)22-13-18-29(19-14-22)15-3-4-25(31)30-16-1-2-17-30/h5-12H,1-4,13-19H2,InChI=1S/C23H28Cl2N4O2/c24-18-5-3-6-19(22(18)25)29-13-4-12-28(14-15-29)11-1-2-16-31-21-10-8-17-7-9-20(30)26-23(17)27-21/h3,5-6,8,10H,1-2,4,7,9,11-16H2,(H,26,27,30),ARIPIPRAZOLE LAUROXIL,MOLINDONE,BICIFADINE,ACARBOSE,ZIPRASIDONE HYDROCHLORIDE,APOMORPHINE HYDROCHLORIDE,CABERGOLINE,InChI=1S/C26H25N3O5S/c1-3-29(19-10-5-4-6-11-19)35(32,33)20-13-14-23-21(15-20)25(30)22(17-27-23)26(31)28-16-18-9-7-8-12-24(18)34-2/h4-15,17H,3,16H2,1-2H3,(H,27,30)(H,28,31),FLIBANSERIN,METOCLOPRAMIDE HYDROCHLORIDE,SIBENADET,CHEMBL1200856,PEROSPIRONE,ZUCLOPENTHIXOL,PRAMIPEXOLE DIHYDROCHLORIDE,AMANTADINE HYDROCHLORIDE,AMANTADINE,InChI=1S/C22H27N5OS/c23-19-9-3-1-7-17(19)22(28)24-11-5-6-12-26-13-15-27(16-14-26)21-18-8-2-4-10-20(18)29-25-21/h1-4,7-10H,5-6,11-16,23H2,(H,24,28),XANOMELINE,LISURIDE,BL-1020,ROTIGOTINE,DARIFENACIN,InChI=1S/C14H19NO3/c1-2-5-15-6-7-17-14-11-8-10(16)3-4-13(11)18-9-12(14)15/h3-4,8,12,14,16H,2,5-7,9H2,1H3/t12-,14-/m0/s1,ETILEVODOPA,PHENOXYBENZAMINE,TANDOSPIRONE,InChI=1S/C19H18F3N3O4S/c1-24-8-10-25(11-9-24)18-14-12-13(29-30(26,27)19(20,21)22)6-7-16(14)28-17-5-3-2-4-15(17)23-18/h2-7,12H,8-11H2,1H3,CHLORPROETHAZINE,CLOTHIAPINE,YOHIMBINE,THIETHYLPERAZINE,FLUPENTIXOL,THIOTHIXENE,ILOPERIDONE,THIOPROPERAZINE,PIZOTYLINE,ESCITALOPRAM,CLEBOPRIDE,HALOPERIDOL,DIHYDROERGOTAMINE,LITHIUM,NEFAZODONE,RACLOPRIDE,ROPINIROLE,LOFEPRAMINE,CYAMEMAZINE,CHEMBL1201732,FLUSPIRILENE,ROPINIROLE HYDROCHLORIDE,ZIPRASIDONE,ITOPRIDE,SARIZOTAN,SERIDOPIDINE,BUPROPION,TRIFLUOPERAZINE HYDROCHLORIDE,DIHYDROERGOCRYPTINE,SULFORIDAZINE,MINAPRINE,DEXMETHYLPHENIDATE,InChI=1S/C25H31NO/c1-24(2,3)25(27)13-14-26-16-21-19-9-5-4-7-17(19)11-12-18-8-6-10-20(23(18)21)22(26)15-25/h4-10,21-22,27H,11-16H2,1-3H3/t21-,2..."
LRRK2,10,"RUXOLITINIB,RUBOXISTAURIN,AXITINIB,BOSUTINIB,NINTEDANIB,FOSTAMATINIB,SUNITINIB,ERLOTINIB,TOFACITINIB,TAMOXIFEN"
SNCA,3,"InChI=1S/Cu,CINPANEMAB,RESVERATROL"


The other genes in that list all have some link to Parkinson's Disease **and** have known chemical modulators.  In some cases, these genes reflect relatively recent findings that may represent novel therapeutic candidates.  For example, BTE found a link between the gene **_LCN2_** and Parkinson's Disease based on both on upregulation in the brains of PD patients, and based on genetic evidence in mouse models ([PMID:27194339](https://www.ncbi.nlm.nih.gov/pubmed/27194339)).  This association was found through [SemMedDB](https://skr3.nlm.nih.gov/SemMedDB/).

<img src="img/lcn2_27194339.png" width="700px" />

Through two other APIs ([mychem.info](https://mychem.info/) and [DGIdb](http://www.dgidb.org/)), BTE discovered six compounds that target **_LCN2_**:

In [8]:
dfFilt.query('node1_name == "LCN2"')[["output_id","output_name"]].drop_duplicates()

,output_id,output_name
16362,drugbank:DB02710,"InChI=1S/C10H11NO6/c12-4-6(10(16)17)11-9(15)5-2-1-3-7(13)8(5)14/h1-3,6,12-14H,4H2,(H,11,15)(H,16,17)/t6-/m0/s1"
29187,chembl:CHEMBL1432,"2,3-DIHYDROXYBENZOIC ACID"
29540,drugbank:DB04476,"InChI=1S/C7H7NO3/c8-7(11)4-2-1-3-5(9)6(4)10/h1-3,9-10H,(H2,8,11)"
40052,drugbank:DB01631,"InChI=1S/C10H20O2/c1-3-4-5-6-7-8-9-10(11)12-2/h3-9H2,1-2H3"
48075,drugbank:DB01926,DB01926
64516,drugbank:DB04043,DB04043


<img src="img/lcn2_summary.png" />